## Medici&oacute;n y An&aacute;lisis de Reacciones Emocionales a Trav&eacute;s de Estimulos Visuales

<style>
    .imagen-ajustada {
        width: 900px;
        height: 280px;
        border-radius: 20px;
    }
</style>

<img src="Emociones.jpg" alt="Una imagen" class="imagen-ajustada">

Las emociones son algo muy b&aacute;sico en el ser humano, sin embargo muchas veces no es necesario expresar la emoci&oacute;n por medio de palabras, sino que con el rostro se puede denotar lo que estas sintiendo. Basandonos en este concepto el proyecto fue planificado para ver si por medio de estimulos visuales, en este caso videos, se pueden obtener reacciones emocionales de acuerdo a lo que se le muestre.

Para el estudio se utilizaron 10 sujetos de prueba, en este caso fueron 6 hombres y 4 mujeres, con un rango de edad de 24 a 45 anos con el fin de ver si generaba algun tipo de reaccion emocional por su parte.

El estudio consistia en mostrar 7 videos de 30 segundos conformados de la siguiente manera:
- Un video de relajacion, con el fin de que el sujeto este en calma.
- Un video donde una nina juega con su perro a que es veterinaria, con el fin de tratar de obtener una reaccion de felicidad o gracia. 
- Un video de relajacion, con el fin de que el sujeto vuelva a un estado neutral.
- Un video donde un hombre golpea a su mujer con un bebe en brazos, con el fin de tratar de obtener una reaccion de enojo, impotencia y rabia.
- Un video de relajacion, con el fin de que el sujeto vuelva a un estado neutral.
- Un video donde una mujer esta abrazando a su perro mientras proceden a dormirlo, con el fin de obtener una reaccion de tristeza.
- Finalmente, un video de relajacion con el fin de que el sujeto vuelva a un estado neutral.      

El fin de comenzar y terminar con un video de relajacion es para ver si hay algun cambio entre el estado inicial y final, como nos indican en la materia de analisis y diseno de experimentos en ciencias de la computacion, tenemos que tener un pre y un post de en el estudio.

## Librerias Importadas

In [ ]:
import os
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Procesamiento de datos

## An&aacute;lisis Exploratorio

## Segmentaci&oacute;n y Extracci&oacute;n de Caracter&iacute;sticas 

## Modelos a Evaluar

## Evaluando los modelos

## Seleccionando el mejor modelo

## Validaci&oacute;n

## Conclusiones